# get_frequently_mutated Final 

In [3]:
import cptac
#en = cptac.Endometrial()
co = cptac.Colon()
#ov = cptac.Ovarian()

import pandas as pd
import numpy as np

In [4]:
def get_frequently_mutated(cancer_object, cutoff = 0.1):  
    """
    Takes a cancer object and find the frequently 
    mutated genes (in the tumor samples) compared to the cutoff.

    Parameters:
    cancer_object (object): cancer type from cptac module 
    cutoff (float): used as a comparison to determine the 
                    status of gene mutation frequency

    Returns:
    freq_mutated_df (pd.DataFrame): DataFrame of frequently 
        mutated genes passing the cutoff. Columns contain the 
        fractions of total unique mutations,missence type 
        mutations, and truncation type mutations per gene.
    
    The Missence_Mut column includes: 
        In_Frame_Del, In_Frame_Ins, Missense_Mutation
   
    The Truncation_Mut column includes: 
        Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, 
        Nonsense_Mutation, Nonstop_Mutation
        
    These columns count multiple mutations of one gene in the 
    same sample, so fractions in the last two columns may 
    exceed the Unique_Samples_Mut column which only counts if 
    the gene was mutated once per sample."""    
    
    # Get total tumors/patients
    omics_and_mutations = cancer_object.join_omics_to_mutations(
        mutations_genes = 'TP53', omics_df_name = 'proteomics', omics_genes = 'TP53')
    tumors = omics_and_mutations.loc[omics_and_mutations['Sample_Status'] == 'Tumor'] 
    total_tumor_samples = len(tumors)
    
    # Get mutations data frame
    somatic_mutations = cancer_object.get_mutations() 
    origin_df = somatic_mutations.reset_index() #prepare to count unique samples
    
    # Drop silent mutations for Ovarian dataset
    if cancer_object.get_cancer_type() == 'ovarian':
        origin_df = origin_df.loc[origin_df['Mutation'] != 'Silent']
        
    # Create two categories in Mutation column - 'M': Missence, 'T': Truncation
    if cancer_object.get_cancer_type() == 'colon':
        missence_truncation_groups = {'frameshift substitution': 'T', 
            'frameshift deletion': 'T', 'frameshift insertion': 'T', 
            'stopgain': 'T', 'stoploss': 'T', 'nonsynonymous SNV': 'M',
            'nonframeshift insertion': 'M','nonframeshift deletion': 'M', 
            'nonframeshift substitution': 'M'}
    else: 
        missence_truncation_groups = {'In_Frame_Del': 'M', 'In_Frame_Ins': 'M',
            'Missense_Mutation': 'M', 'Frame_Shift_Del': 'T','Nonsense_Mutation': 'T', 
            'Splice_Site': 'T', 'Frame_Shift_Ins': 'T','Nonstop_Mutation':'T'}
    mutations_replaced_M_T = origin_df.replace(missence_truncation_groups)
    
    # Find frequently mutated genes (total fraction > cutoff)
    # Step 1 - group by gene and count unique samples
    count_mutations = origin_df.groupby(['Gene']).nunique()
    # Step 2 - format
    count_mutations = count_mutations.rename(columns={"Sample_ID": "Unique_Samples_Mut"})
    count_mutations = count_mutations.drop(['Gene', 'Mutation','Location'], axis = 1)
    # Step 3 - filter using the cutoff and create fraction
    fraction_mutated = count_mutations.apply(lambda x: x / total_tumor_samples)
    fraction_greater_than_cutoff = fraction_mutated.where(lambda x: x > cutoff) #na used when false
    filtered_gene_df = fraction_greater_than_cutoff.dropna()
    
    # Create and join Missence column (following similar steps as seen above)
    miss = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'M']
    count_miss = miss.groupby(['Gene']).nunique()
    missence_df = count_miss.rename(columns={"Sample_ID": "Missence_Mut"})
    missence_df = missence_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)
    fraction_missence = missence_df.apply(lambda x: x / total_tumor_samples)
    freq_mutated_df = filtered_gene_df.join(fraction_missence, how='left').fillna(0)
    
    
    # Create and join Truncation column (following similar steps as seen above)
    trunc = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'T']
    count_trunc = trunc.groupby(['Gene']).nunique()
    truncation_df = count_trunc.rename(columns={"Sample_ID": "Truncation_Mut"})
    truncation_df = truncation_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)
    fraction_truncation = truncation_df.apply(lambda x: x / total_tumor_samples)
    freq_mutated_df = freq_mutated_df.join(fraction_truncation, how='left').fillna(0)

    return freq_mutated_df

In [10]:
df = get_frequently_mutated(co, .18)
df.reset_index()

,Gene,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
0,ABCA13,0.195876,0.164948,0.103093
1,ACVR2A,0.257732,0.020619,0.257732
2,ADGRV1,0.195876,0.134021,0.092784
3,AHNAK2,0.216495,0.195876,0.030928
4,APC,0.845361,0.113402,0.824742
5,APOB,0.195876,0.175258,0.061856
6,ARID1A,0.237113,0.072165,0.185567
7,BRAF,0.185567,0.164948,0.030928
8,CASP5,0.195876,0.000000,0.195876
9,CCDC168,0.298969,0.195876,0.195876


In [45]:
df = get_frequently_mutated(en, .2)
df

,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
Gene,,,
ARID1A,0.452632,0.136842,0.400000
CTCF,0.284211,0.094737,0.242105
CTNNB1,0.305263,0.305263,0.000000
KMT2B,0.242105,0.115789,0.126316
KRAS,0.326316,0.326316,0.000000
PIK3CA,0.494737,0.484211,0.010526
PIK3R1,0.389474,0.231579,0.189474
PTEN,0.789474,0.463158,0.568421
TP53,0.221053,0.157895,0.073684


In [46]:
df = get_frequently_mutated(ov, .2)
df

,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
Gene,,,
MUC4,0.325301,0.313253,0.012048
TP53,0.927711,0.602410,0.325301
TTN,0.301205,0.265060,0.084337


In [48]:
#test results
m = co.get_mutations()

#m = m.loc[m['Mutation'] != 'Silent']
gene = 'CASP5'
g = m.loc[m['Gene'] == gene]
#print(g)
print(len(g.index.unique()))

r = g.groupby('Mutation')

r.groups


19


{'frameshift deletion': Index(['S055', 'S055', 'S055', 'S055', 'S074', 'S074', 'S074', 'S080', 'S080',
        'S080', 'S081', 'S081'],
       dtype='object', name='Sample_ID'),
 'frameshift insertion': Index(['S031', 'S031', 'S031', 'S031'], dtype='object', name='Sample_ID'),
 'frameshift substitution': Index(['S006', 'S006', 'S006', 'S009', 'S009', 'S009', 'S009', 'S009', 'S009',
        'S014', 'S014', 'S014', 'S018', 'S018', 'S018', 'S018', 'S018', 'S018',
        'S023', 'S023', 'S023', 'S023', 'S023', 'S023', 'S023', 'S023', 'S028',
        'S028', 'S028', 'S028', 'S030', 'S030', 'S030', 'S033', 'S033', 'S033',
        'S033', 'S035', 'S035', 'S035', 'S035', 'S035', 'S052', 'S052', 'S052',
        'S052', 'S052', 'S052', 'S052', 'S052', 'S052', 'S052', 'S055', 'S055',
        'S055', 'S055', 'S064', 'S064', 'S064', 'S065', 'S065', 'S065', 'S065',
        'S065', 'S065', 'S065', 'S065', 'S074', 'S074', 'S074', 'S080', 'S080',
        'S080', 'S101', 'S101', 'S101', 'S108', 'S108',